### Trade Demo

#### Goal: 
- Load the trade data for the country `Canada`
- Launch a domain node for canada
- Login into the domain node
- Format the `Canada` trade dataset and convert to Numpy array
- Convert the dataset to a private tensor
- Upload `Canada's` trade on the domain node
- Create a Data Scientist User

In [2]:
%load_ext autoreload
%autoreload 2

import pandas as pd

canada = pd.read_csv("../notebooks/trade_demo/datasets/ca - feb 2021.csv")

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


/home/user/anaconda3/envs/adp/lib/python3.9/site-packages/IPython/core/interactiveshell.py:3441: DtypeWarning: Columns (14) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


### Step 1: Load the dataset

We have trade data for the country, which has provided data from Feb 2021. They key colums are:

- Commodity Code: the official code of that type of good
- Reporter: the country claiming the import/export value
- Partner: the country being claimed about
- Trade Flow: the direction of the goods being reported about (imports, exports, etc)
- Trade Value (US$): the declared USD value of the good

Let's have a quick look at the top five rows of the dataset.

In [3]:
canada.head()

,Classification,Year,Period,Period Desc.,Aggregate Level,Is Leaf Code,Trade Flow Code,Trade Flow,Reporter Code,Reporter,...,Partner,Partner ISO,Commodity Code,Commodity,Qty Unit Code,Qty Unit,Qty,Netweight (kg),Trade Value (US$),Flag
0,HS,2021,202102,February 2021,4,0,1,Imports,124,Canada,...,"Other Asia, nes",NaN,6117,"Clothing accessories; made up, knitted or croc...",0,NaN,NaN,NaN,9285,0
1,HS,2021,202102,February 2021,2,0,1,Imports,124,Canada,...,Egypt,NaN,18,Cocoa and cocoa preparations,0,NaN,NaN,0.0,116604,0
2,HS,2021,202102,February 2021,2,0,1,Imports,124,Canada,...,United Kingdom,NaN,18,Cocoa and cocoa preparations,0,NaN,NaN,0.0,1495175,0
3,HS,2021,202102,February 2021,2,0,1,Imports,124,Canada,...,United Rep. of Tanzania,NaN,18,Cocoa and cocoa preparations,0,NaN,NaN,0.0,2248,0
4,HS,2021,202102,February 2021,2,0,1,Imports,124,Canada,...,Singapore,NaN,18,Cocoa and cocoa preparations,0,NaN,NaN,0.0,47840,0


### Step 2: Spin up the Domain Node (if you haven't already)

SKIP THIS STEP IF YOU'VE ALREADY RUN IT!!!

As the main requirement of this demo is to perform analysis on the Canada's trade dataset. So, we need to spin up a domain node for Canada.

Assuming you have [Docker](https://www.docker.com/) installed and configured with >=8GB of RAM, navigate to PySyft/packages/hagrid and run the following commands in separate terminals (can be done at the same time):


```bash
# install hagrid cli tool
pip install -e .
```

```bash
hagrid launch Canada domain
```

<div class="alert alert-block alert-info">
    <b>Quick Tip:</b> Don't run this now, but later when you want to stop these nodes, you can simply run the same argument with the "stop" command. So from the PySyft/grid directory you would run. Note that these commands will delete the database by default. Add the flag "--keep_db=True" to keep the database around. Also note that simply killing the thread created by ./start is often insufficient to actually stop all nodes. Run the ./stop script instead. To stop the nodes listed above (and delete their databases) run:

```bash
hagrid land Canada
```
</div>

### Step 3: Login into the Domain as the Admin User

In [5]:
import syft as sy

# Let's login into the domain node
domain_node = sy.login(email="info@openmined.org", password="changethis", port=8081)

Connecting to http://localhost:8081... done! 	 Logging into canada... done!


In [6]:
canada.head()

,Classification,Year,Period,Period Desc.,Aggregate Level,Is Leaf Code,Trade Flow Code,Trade Flow,Reporter Code,Reporter,...,Partner,Partner ISO,Commodity Code,Commodity,Qty Unit Code,Qty Unit,Qty,Netweight (kg),Trade Value (US$),Flag
0,HS,2021,202102,February 2021,4,0,1,Imports,124,Canada,...,"Other Asia, nes",NaN,6117,"Clothing accessories; made up, knitted or croc...",0,NaN,NaN,NaN,9285,0
1,HS,2021,202102,February 2021,2,0,1,Imports,124,Canada,...,Egypt,NaN,18,Cocoa and cocoa preparations,0,NaN,NaN,0.0,116604,0
2,HS,2021,202102,February 2021,2,0,1,Imports,124,Canada,...,United Kingdom,NaN,18,Cocoa and cocoa preparations,0,NaN,NaN,0.0,1495175,0
3,HS,2021,202102,February 2021,2,0,1,Imports,124,Canada,...,United Rep. of Tanzania,NaN,18,Cocoa and cocoa preparations,0,NaN,NaN,0.0,2248,0
4,HS,2021,202102,February 2021,2,0,1,Imports,124,Canada,...,Singapore,NaN,18,Cocoa and cocoa preparations,0,NaN,NaN,0.0,47840,0


In [7]:
# For, simplicity we will upload the first 10000 rows of the dataset.
canada = canada[:10000]

### Step 4: Format dataset and convert to numpy array

In [8]:
# In order to the convert the whole dataset into an numpy array,
# We need to format string to integer values.

In [9]:
# Let's create a function that converts string to int.

import hashlib
from math import isnan, nan

hash_db = {}
hash_db[nan] = nan


def convert_string(s: str, digits: int = 15):
    """Maps a string to a unique hash using SHA, converts it to a hash or an int"""
    if type(s) is str:
        new_hash = int(hashlib.sha256(s.encode("utf-8")).hexdigest(), 16) % 10 ** digits
        hash_db[s] = new_hash
        return new_hash
    else:
        return s

In [26]:
# Let's filter out the string/object type columns
string_cols = []
for col, dtype in canada.dtypes.items():
    if dtype in ['object', 'str']:
        string_cols.append(col)

# Convert string values to integer
for col in canada.columns:
    canada[col] = canada[col].map(lambda x: convert_string(x))

In [27]:
# Let's checkout the formatted dataset
canada.head()

,Classification,Year,Period,Period Desc.,Aggregate Level,Is Leaf Code,Trade Flow Code,Trade Flow,Reporter Code,Reporter,...,Partner,Partner ISO,Commodity Code,Commodity,Qty Unit Code,Qty Unit,Qty,Netweight (kg),Trade Value (US$),Flag
0,109781654799833,2021,202102,618044004745978,4,0,1,740486968595500,124,524003986429176,...,251096594821383,NaN,20063876956541,6.602623e+14,0,NaN,NaN,NaN,9285,0
1,109781654799833,2021,202102,618044004745978,2,0,1,740486968595500,124,524003986429176,...,382796968671830,NaN,897295160791370,4.288560e+14,0,NaN,NaN,0.0,116604,0
2,109781654799833,2021,202102,618044004745978,2,0,1,740486968595500,124,524003986429176,...,37977140198169,NaN,897295160791370,4.288560e+14,0,NaN,NaN,0.0,1495175,0
3,109781654799833,2021,202102,618044004745978,2,0,1,740486968595500,124,524003986429176,...,711459702489058,NaN,897295160791370,4.288560e+14,0,NaN,NaN,0.0,2248,0
4,109781654799833,2021,202102,618044004745978,2,0,1,740486968595500,124,524003986429176,...,718934534792483,NaN,897295160791370,4.288560e+14,0,NaN,NaN,0.0,47840,0


In [33]:
# Great !!! now let's convert the whole dataset to numpy array.
np_dataset = canada.values

# Type cast to float values to prevent overflow
np_dataset = np_dataset.astype(float) 

### Step 5: Converting the dataset to private tensors

In [29]:
from syft.core.adp.entity import Entity

In [31]:
# The 'Partner' column i.e the countries to which the data is exported
# is private, therefore let's create entities for each of the partner defined

entities = [Entity(name=partner) for partner in canada["Partner"]]

In [46]:
# Let's convert the whole dataset to a private tensor

private_dataset_tensor = sy.Tensor(np_dataset).private(0.01, 1e15, entity=Entity(name="Canada")).tag("private_canada_trade_dataset")

### Step 6: Upload Canada's trade data on the domain

In [47]:
# Awesome, now let's upload the dataset to the domain.
# For, simplicity we will upload the first 10000 rows of the dataset.

domain_node.load_dataset(
    assets={"feb2020": private_dataset_tensor},
    name="Canada Trade Data - First 10000 rows",
    description="""A collection of reports from Canada's statistics 
                    bureau about how much it thinks it imports and exports from other countries.""",
)

Cool !!! The dataset was successfully uploaded onto the domain.

In [43]:
# Now, let's check datasets available on the domain.
domain_node.datasets.pandas()

,str_metadata,description,tags,name,blob_metadata,id,data,manifest
0,b'\n\x14syft.lib.python.Dict\x12\x14\x1a\x12\n...,b'\n\x16syft.lib.python.String\x12\xab\x01\n\x...,b'\n\x19syft.lib.python.list.List\x12\x16\x12\...,b'\n\x16syft.lib.python.String\x12)\n\x11Canad...,"b""\n\x14syft.lib.python.Dict\x12\x14\x1a\x12\n...",b'\n\x16syft.lib.python.String\x12<\n$1192bcd5...,b'\n\x19syft.lib.python.list.List\x12\xa1\x04\...,b'\n\x16syft.lib.python.String\x12\x16\x12\x12...
1,b'\n\x14syft.lib.python.Dict\x12\x14\x1a\x12\n...,b'\n\x16syft.lib.python.String\x12\xab\x01\n\x...,b'\n\x19syft.lib.python.list.List\x12\x16\x12\...,b'\n\x16syft.lib.python.String\x12)\n\x11Canad...,b'\n\x14syft.lib.python.Dict\x12\x14\x1a\x12\n...,"b""\n\x16syft.lib.python.String\x12<\n$269043c0...",b'\n\x19syft.lib.python.list.List\x12\xa1\x04\...,"b""\n\x16syft.lib.python.String\x12\x16\x12\x12..."
2,b'\n\x14syft.lib.python.Dict\x12\x14\x1a\x12\n...,b'\n\x16syft.lib.python.String\x12\xab\x01\n\x...,"b""\n\x19syft.lib.python.list.List\x12\x16\x12\...",b'\n\x16syft.lib.python.String\x12)\n\x11Canad...,b'\n\x14syft.lib.python.Dict\x12\x14\x1a\x12\n...,b'\n\x16syft.lib.python.String\x12<\n$3789e774...,b'\n\x19syft.lib.python.list.List\x12\xa1\x04\...,b'\n\x16syft.lib.python.String\x12\x16\x12\x12...
3,b'\n\x14syft.lib.python.Dict\x12\x14\x1a\x12\n...,"b""\n\x16syft.lib.python.String\x12\xab\x01\n\x...",b'\n\x19syft.lib.python.list.List\x12\x16\x12\...,b'\n\x16syft.lib.python.String\x12)\n\x11Canad...,b'\n\x14syft.lib.python.Dict\x12\x14\x1a\x12\n...,b'\n\x16syft.lib.python.String\x12<\n$cc28df5f...,b'\n\x19syft.lib.python.list.List\x12\xa3\x04\...,b'\n\x16syft.lib.python.String\x12\x16\x12\x12...
4,b'\n\x14syft.lib.python.Dict\x12\x14\x1a\x12\n...,"b""\n\x16syft.lib.python.String\x12\xab\x01\n\x...",b'\n\x19syft.lib.python.list.List\x12\x16\x12\...,b'\n\x16syft.lib.python.String\x12)\n\x11Canad...,b'\n\x14syft.lib.python.Dict\x12\x14\x1a\x12\n...,b'\n\x16syft.lib.python.String\x12<\n$6a49b4ed...,b'\n\x19syft.lib.python.list.List\x12\xa2\x04\...,b'\n\x16syft.lib.python.String\x12\x16\x12\x12...
5,b'\n\x14syft.lib.python.Dict\x12\x14\x1a\x12\n...,"b""\n\x16syft.lib.python.String\x12\xab\x01\n\x...",b'\n\x19syft.lib.python.list.List\x12\x16\x12\...,b'\n\x16syft.lib.python.String\x12;\n#Canada T...,b'\n\x14syft.lib.python.Dict\x12\x14\x1a\x12\n...,b'\n\x16syft.lib.python.String\x12<\n$524c9e2b...,"b""\n\x19syft.lib.python.list.List\x12\xa4\x04\...",b'\n\x16syft.lib.python.String\x12\x16\x12\x12...
6,b'\n\x14syft.lib.python.Dict\x12\x14\x1a\x12\n...,"b""\n\x16syft.lib.python.String\x12\xab\x01\n\x...",b'\n\x19syft.lib.python.list.List\x12\x16\x12\...,b'\n\x16syft.lib.python.String\x12;\n#Canada T...,b'\n\x14syft.lib.python.Dict\x12\x14\x1a\x12\n...,b'\n\x16syft.lib.python.String\x12<\n$ff32d492...,"b""\n\x19syft.lib.python.list.List\x12\xa4\x04\...",b'\n\x16syft.lib.python.String\x12\x16\x12\x12...
7,b'\n\x14syft.lib.python.Dict\x12\x14\x1a\x12\n...,"b""\n\x16syft.lib.python.String\x12\xab\x01\n\x...",b'\n\x19syft.lib.python.list.List\x12\x16\x12\...,b'\n\x16syft.lib.python.String\x12;\n#Canada T...,b'\n\x14syft.lib.python.Dict\x12\x14\x1a\x12\n...,b'\n\x16syft.lib.python.String\x12<\n$75bf69bc...,b'\n\x19syft.lib.python.list.List\x12\xa4\x04\...,b'\n\x16syft.lib.python.String\x12\x16\x12\x12...
8,"b""\n\x14syft.lib.python.Dict\x12\x14\x1a\x12\n...","b""\n\x16syft.lib.python.String\x12\xab\x01\n\x...",b'\n\x19syft.lib.python.list.List\x12\x16\x12\...,b'\n\x16syft.lib.python.String\x12<\n$Canada T...,b'\n\x14syft.lib.python.Dict\x12\x14\x1a\x12\n...,b'\n\x16syft.lib.python.String\x12<\n$069eedd0...,b'\n\x19syft.lib.python.list.List\x12\xa5\x04\...,b'\n\x16syft.lib.python.String\x12\x16\x12\x12...
9,b'\n\x14syft.lib.python.Dict\x12\x14\x1a\x12\n...,"b""\n\x16syft.lib.python.String\x12\xab\x01\n\x...",b'\n\x19syft.lib.python.list.List\x12\x16\x12\...,b'\n\x16syft.lib.python.String\x12<\n$Canada T...,b'\n\x14syft.lib.python.Dict\x12\x14\x1a\x12\n...,b'\n\x16syft.lib.pyth

### Step 7: Create a Data Scientist User

Open http://localhost:8081, login is the root user (username: info@openmined.org, password:changethis), and create a user with the following attributes:

- Name: Sheldon Cooper
- Email: sheldon@caltech.edu
- Password: bazinga

In [10]:
# Alternatively, you can create the same from the notebook itself.
domain_node.users.create(
    **{
        "name": "Sheldon Cooper",
        "email": "sheldon@caltech.edu",
        "password": "bazinga",
    },
)

```
Great !!! We were successfully able to create a new user.
Now, let's move to the Data Scientist notebook, to check out their experience.
```

### Step 8: Decline request to download entire datsaet

In [49]:
# Let's check if there are any requests pending for approval.
domain_node.requests.pandas

,Requested Object's tags,Reason,Request ID,Requested Object's ID,Requested Object's type
0,[#feb2020],Access whole dataset,<UID: 992e2437877b4de5a04a4d2e44363235>,<UID: 2be294e4b63047738ff0d484f726d115>,<class 'syft.core.tensor.tensor.Tensor'>


In [50]:
# Looks like the DS wants to download the whole dataset. We cannot allow that.
# Let's select and deny this request.
domain_node.requests[0].deny()

### STOP: Return to Data Scientist - Canada.ipynb - STEP 3!!